In [22]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from dotenv import load_dotenv
from parsel import Selector
import urllib.request
import time
import re
import os

In [23]:
# Load .env file
load_dotenv(verbose=True)

# specifies the path to the chromedriver.exe

def initWebdriver():
    driver = webdriver.Chrome("C:/Users/WalidCorleone/selenium/DeepAnalyzerImages/Scraper/chromedriver.exe")
    
    # Maximaze page size
    driver.maximize_window()
    # driver.get method() will navigate to a page given by the URL address
    driver.get('https://www.linkedin.com')
    try:
        # locate email form by_ID
        username = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.ID, 'session_key')))
        # send_keys() to simulate key strokes
        username.send_keys("khirdine.walid@gmail.com")
        time.sleep(0.5)
        # locate password form by_ID
        password = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.ID, 'session_password')))
        # send_keys() to simulate key strokes
        password.send_keys("LINKDIN2lkref6w@")
     # Locate submit button by_xpath
        sign_in_button = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.XPATH, '//*[@type="submit"]')))
        # Signin button click
        sign_in_button.click()
    except NoSuchElementException:
        print("\tError finding MessagingList")
        pass
    return driver


In [33]:
# get source code of my networks profiles and scroll 30 times 
def networkSource(link):
    driver.get(link)
    try:
        time.sleep(3)
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((
            By.XPATH, "//button/li-icon[@type='chevron-down-icon']"))).click()
        # Scrolling with chrono of 5 seconds ( you can changes as you like)
        time_end = time.time() + 10
        while time.time() < time_end:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1.5)
            sel = Selector(text=driver.page_source)
    except NoSuchElementException:
        print("\tError finding MessagingList")
        pass
    return sel

In [25]:
# Get src of each linkedin image profile 

def linkedinImgSrc():
    sel = networkSource('https://www.linkedin.com/mynetwork/')
    # to get all the link in the script a
    image_profile= set([a.attrib['src'] 
                        for a in sel.css('img') 
                    if ("profile-displayphoto-shrink" or 
                        "profile-framedphoto-shrink"  in a.attrib['src'])])
    print("image profile ", image_profile)
    return image_profile

In [5]:
# Get the urls from local and check if they already exists in href got

def linkToDownload():
    links = linkedinImgSrc()
    linktoload = [row for row in links if row not in linksStored()]
    
    print("Les profile a télécharger <<<<<<<<<.......................>>>>>>>", linktoload)
    print("Local link <------>>>>>>", linksStored())
    return linktoload

In [6]:
# Get the links from storelink

def linksStored():
    with open('imagename.txt', "r") as local:
        local_links = local.readlines()
        # to remove the \n
        final_link = [local.strip("\n") for local in local_links]
        local.close()
    return final_link

In [7]:
# Adding links  in local

def StoreImgName(links):
    with open('imagename.txt', 'a') as f:
        f.write(links+" \n")
        f.close()
    return links

In [12]:
def GetImageName(src):
    #name=[item for nom in src for item in re.findall("(\/0\/[A-z0-9_-]*)+\/?", nom)]
    name=re.findall("(a&.=..[A-z0-9_-]*)", src)
    return name

In [27]:
  def downloadImages(image_url,image_name):
    try:
        # Getting our list of images
        local_image_names = linksStored()
        # Compare the name of local images and the futur downloading images to not have a double
        for name in image_name:
            if name+'.png' not in local_image_names:
                # Download the image
                urllib.request.urlretrieve(
                    image_url, "images/"+name+".png")
                # Add the link to the local txt
                StoreImgName(image_url)
                print("Download successfull, the link downloaded",image_url,'\n')
                # add a 5 second pause loading each URL
    except urllib.error.HTTPError as e:
        print(e.code)
        print(e.read())
        pass

In [28]:
# DownLoad function to get profile images

def downloadImageAndLink(driver):
    try:
        for image_url in linkToDownload():
            # get the source of the name profile
            print(">>>>>>>>>>>>>>>>>>>>>>>>>>>> image URL <<<<<<<<<<<<<<<<<",image_url)
            image_name=GetImageName(image_url)
            print(">>>>>>>>>>>>>>>>>>>>>>>>>>>image NAME<<<<<<<<<<<<<<<<<<<<<<<<<<<<",image_name)
            downloadImages(image_url,image_name)
    except NoSuchElementException:
        print("\tError finding MessagingList")
        pass
    driver.close()

In [30]:
if __name__ == "__main__":
    driver = initWebdriver()
    downloadImageAndLink(driver)

C:\Users\WalidCorleone\AppData\Local\Temp\ipykernel_28576\58734777.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:/Users/WalidCorleone/selenium/DeepAnalyzerImages/Scraper/chromedriver.exe")


image profile  {'https://media-exp1.licdn.com/dms/image/C5603AQEJLnT9t0KhKQ/profile-displayphoto-shrink_200_200/0/1538418516569?e=1658966400&v=beta&t=TxU5BK4F4RSvY4MbUYEOXoevOXoHi_Stcwe6x3wDrOQ', 'https://media-exp1.licdn.com/dms/image/C4D16AQG-iV7zY3WPHw/profile-displaybackgroundimage-shrink_200_800/0/1650830714088?e=1658966400&v=beta&t=VB9T5CZ6wdxsHADkboo6VdBMJ1X5qV0fk5VW1KGRWUM', 'https://media-exp1.licdn.com/dms/image/C4D16AQGXqQPsSmjzCA/profile-displaybackgroundimage-shrink_200_800/0/1571772385220?e=1658966400&v=beta&t=081Fqe9pNpIu0QAgy8khVrDMx-ehMGKHs2vIUosJh90', 'https://media-exp1.licdn.com/dms/image/D4D35AQHdPOH9SZQLAQ/profile-framedphoto-shrink_200_200/0/1641378280330?e=1653908400&v=beta&t=lpQlsaFRANyDk-4COv9tsMe8fqlLN2gQw0hmYUQwzv0', 'https://media-exp1.licdn.com/dms/image/C4E03AQHMaz8qC5rAuQ/profile-displayphoto-shrink_200_200/0/1618930317365?e=1658966400&v=beta&t=uazvQHjUoGzxhmGtRvCo9CIuzmV7ON1z5DKm_nePTD0', 'https://media-exp1.licdn.com/dms/image/C5603AQE8g0IXJF3cow/profi

FileNotFoundError: [Errno 2] No such file or directory: 'imagename.txt'

In [13]:
GetImageName(' https://media-exp1.licdn.com/dms/image/C4D16AQESP17_6BscFw/profile-displaybackgroundimage-shrink_200_800/0/1644321863654?e=1658361600&v=beta&t=xr1mSgxmWXqw23O4GiwO4moxl7tmSO-USU4UzFXrHP0 ')

['a&t=xr1mSgxmWXqw23O4GiwO4moxl7tmSO-USU4UzFXrHP0']

In [40]:
im=["https://media-exp1.licdn.com/dms/image/C4E03AQHs8om8Og7BhQ/profile-displayphoto-shrink_200_200/0/1593094960825?e=1658361600&v=beta&t=3fM8otrvAvVaVnL3w76_8YAA5IFnPctz3TBlGoQD_lM","https://media-exp1.licdn.com/dms/image/C4E03AQHYF75fCfEVbw/profile-displayphoto-shrink_200_200/0/1650277731323?e=1658361600&v=beta&t=5ecBt6K-cqhCjTvnagcP7N6ouz4SK4hv6bbevNuq6bE","https://media-exp1.licdn.com/dms/image/C4E03AQHs8om8Og7BhQ/profile-displayphoto-shrink_200_200/0/1593094960825?e=1658361600&v=beta&t=3fM8otrvAvVaVnL3w76_8YAA5IFnPctz3TBlGoQD_lM","https://media-exp1.licdn.com/dms/image/C4D16AQEwTvdHaM0Fqw/profile-displaybackgroundimage-shrink_200_800/0/1599053629710?e=1658361600&v=beta&t=JxgO-26g8Sit4U5SNzLtX-OKsq-S1zTc1oYQTtGT2Vg"]
if "a&t=5ecBt6K-cqhCjTvnagcP7N6ouz4SK4hv6bbevNuq6bE" in im:
    print("true")
else:
    print("false")

false


[['a&t=3fM8otrvAvVaVnL3w76_8YAA5IFnPctz3TBlGoQD_lM'], ['a&t=5ecBt6K-cqhCjTvnagcP7N6ouz4SK4hv6bbevNuq6bE'], ['a&t=3fM8otrvAvVaVnL3w76_8YAA5IFnPctz3TBlGoQD_lM'], ['a&t=JxgO-26g8Sit4U5SNzLtX-OKsq-S1zTc1oYQTtGT2Vg']]


In [34]:
linkedinImgSrc()

image profile  {'https://media-exp1.licdn.com/dms/image/C4D03AQFFfCJ8IbcGbw/profile-displayphoto-shrink_200_200/0/1596628275847?e=1658966400&v=beta&t=GlPalSlG08U6unadh-nUZdBun22NvyhWu-5RRDClgak', 'https://media-exp1.licdn.com/dms/image/D4D35AQGuWbVstpaCMg/profile-framedphoto-shrink_200_200/0/1636641840617?e=1653908400&v=beta&t=HMPuzX24HHY5a49Cn1wdKXQYTeLGp2nodKPCMau1Ezc', 'https://media-exp1.licdn.com/dms/image/C4D03AQGXK_snSdUuXw/profile-displayphoto-shrink_200_200/0/1592070871170?e=1658966400&v=beta&t=ruhrcgeT4qRfTDvIkC0ZJMSZvlGODIucIkbDkHBC8Cg', 'https://media-exp1.licdn.com/dms/image/C4D03AQHEWYYrzzBiOA/profile-displayphoto-shrink_200_200/0/1601296636615?e=1658966400&v=beta&t=pCCZE9ObuqwJ2YLrPR5WwOyb15mD0p4xxQ6q-WPGrVQ', 'https://media-exp1.licdn.com/dms/image/C5616AQF0UlplAEYrXA/profile-displaybackgroundimage-shrink_200_800/0/1586808367019?e=1658966400&v=beta&t=zqZhua5oWYB7L0IB3PRj1W0Dom6NQbyEGqHIdJUwmPI', 'https://media-exp1.licdn.com/dms/image/D4E1EAQG4-EYb82GeUA/event-backgroun

{'data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7',
 'https://media-exp1.licdn.com/dms/image/C4D03AQE-aYZiXdnRMA/profile-displayphoto-shrink_200_200/0/1621952042271?e=1658966400&v=beta&t=jZ7etSqu6zKhT4IxfWBjCJQHPDGk6I4Ac1rikjO2jsU',
 'https://media-exp1.licdn.com/dms/image/C4D03AQE36pAPm6cjTA/profile-displayphoto-shrink_200_200/0/1612089606195?e=1658966400&v=beta&t=dlRqA6kl164DaOMul5tIMlR5J6Rv8pAk0UFl5jxshvc',
 'https://media-exp1.licdn.com/dms/image/C4D03AQE4P2T-uneJkw/profile-displayphoto-shrink_200_200/0/1610408622741?e=1658966400&v=beta&t=vgNOtKd72EGxWw4lDtLygs0aGQKu1c4vnJ6FdpH_R_o',
 'https://media-exp1.licdn.com/dms/image/C4D03AQEJVSi-vGzDnQ/profile-displayphoto-shrink_200_200/0/1632780051799?e=1658966400&v=beta&t=G5gIr1el01YLziAvqXDo-QyMQlDc_Y6X7NgJXaSoVI0',
 'https://media-exp1.licdn.com/dms/image/C4D03AQEKjW20VKlfbA/profile-displayphoto-shrink_200_200/0/1604331814788?e=1658966400&v=beta&t=jY_ce3p2-2ne5TO3Gpm4DUYoSs7Co1yTV_rYIosbZRg',
 'https://me

In [42]:
local_link=linksStored()
s=[GetImageName(local_link) for local in local_link]

FileNotFoundError: [Errno 2] No such file or directory: 'imagename.txt'